In [2]:
import numpy as np
import random

In [3]:
import pandas as pd
import ast

In [4]:
from gensim.models import KeyedVectors
from gensim.test.utils import datapath

In [171]:
word_vectors = KeyedVectors.load_word2vec_format('../data/embedding/rdf2vec_vectors_15_06_2021.txt', binary=False)

# C text format

In [172]:
word_vectors.most_similar('http://idea.rpi.edu/heals/kb/usda#01003', topn=5)

[('http://idea.rpi.edu/heals/kb/usda#04042', 0.9453282356262207),
 ('http://idea.rpi.edu/heals/kb/usda#04585', 0.9446382522583008),
 ('http://idea.rpi.edu/heals/kb/usda#18248', 0.9445690512657166),
 ('http://idea.rpi.edu/heals/kb/usda#01236', 0.944229245185852),
 ('http://idea.rpi.edu/heals/kb/usda#42135', 0.9411625862121582)]

In [176]:
df= pd.read_csv('../data/usda_nutriscores.csv')
df.head()

,Unnamed: 0,foods,labels,ingredients,nutri_values,nutri_scores
0,0,http://idea.rpi.edu/heals/kb/usda#01001,"BUTTER,WITH SALT",['http://www.w3id.org/foodkg/usda#GmWt_1-X_wFU...,19,E
1,1,http://idea.rpi.edu/heals/kb/usda#01002,"BUTTER,WHIPPED,WITH SALT",['http://www.w3id.org/foodkg/usda#GmWt_1-qmUAd...,19,E
2,2,http://idea.rpi.edu/heals/kb/usda#01003,"BUTTER OIL,ANHYDROUS",['http://www.w3id.org/foodkg/usda#GmWt_1-JG6dQ...,12,D
3,3,http://idea.rpi.edu/heals/kb/usda#01004,"CHEESE,BLUE",['http://www.w3id.org/foodkg/usda#GmWt_1-3aSX_...,16,D
4,4,http://idea.rpi.edu/heals/kb/usda#01005,"CHEESE,BRICK",['http://www.w3id.org/foodkg/usda#GmWt_1-hPYzg...,12,D


In [177]:
df= df[['foods', 'nutri_values']]

In [178]:
nutriscore ={}
for i, row in df.iterrows():
    nutriscore[row['foods']] = row['nutri_values']

In [179]:
list(nutriscore.items())[:10]

[('http://idea.rpi.edu/heals/kb/usda#01001', 19),
 ('http://idea.rpi.edu/heals/kb/usda#01002', 19),
 ('http://idea.rpi.edu/heals/kb/usda#01003', 12),
 ('http://idea.rpi.edu/heals/kb/usda#01004', 16),
 ('http://idea.rpi.edu/heals/kb/usda#01005', 12),
 ('http://idea.rpi.edu/heals/kb/usda#01006', 11),
 ('http://idea.rpi.edu/heals/kb/usda#01007', 14),
 ('http://idea.rpi.edu/heals/kb/usda#01008', 13),
 ('http://idea.rpi.edu/heals/kb/usda#01009', 13),
 ('http://idea.rpi.edu/heals/kb/usda#01010', 13)]

In [159]:
def get_simscore_ingrank_onlyss_multisamerank(fromt, asdf):
    # return rank based on KG embedding +revised nutriscore
    TOPK=500
    mostSimilar =word_vectors.most_similar(fromt, topn=TOPK)
    for i,(e, sim) in enumerate(mostSimilar):
        if e == asdf:
            return i+1    
    return TOPK

In [150]:
def get_simscore_ingrank_kg_nutriscore_multisamerank(fromt, asdf):
    # return rank based on KG embedding +revised nutriscore
    TOPK=20
    mostSimilar =word_vectors.most_similar(fromt, topn=TOPK)
    nutriscore_ranking= [ (e,nutriscore[e]) for (e, sim) in mostSimilar if e in nutriscore]
    nutriscore_ranking= sorted(nutriscore_ranking, key= lambda x:x[1])
    #print (nutriscore_ranking)
    for i,(e, sim) in enumerate(nutriscore_ranking):
        if e == asdf:
            return i+1    
    return TOPK

In [151]:
def mrr_map_new(scraped_subs_dict, opt = 1):
    rank_scores = []
    ave_p = []
    in_top_5 = 0
    in_top_10 = 0
    print('number of ings: ',len(scraped_subs_dict.keys()))

    for fromt in scraped_subs_dict.keys():
        relevant_ranks = []
        min_rank = 9999999999999999
        for asdf in scraped_subs_dict[fromt]:
            if opt == 1:
                rank = get_simscore_ingrank_onlyss_multisamerank(fromt, asdf)
            elif opt == 2:
                rank = get_simscore_ingrank_kg_nutriscore_multisamerank(fromt, asdf)   
            relevant_ranks.append(rank)
            if rank < min_rank:
                min_rank = rank
        rank = min_rank

        rank_scores.append(1.0/rank)
        if min_rank <= 5:
            in_top_5 += 1
        if min_rank <= 10:
            in_top_10 += 1

        precisions = []
        for rank in relevant_ranks:
            good_docs = len([r for r in relevant_ranks if r <= rank])
            precisions.append(good_docs/rank)
        if len(precisions) == 0:
            precisions = [0]
        ave_p.append(np.mean(precisions))

    print('mean reciprocal rank: ', np.mean(rank_scores))
    print('recallrate at 5: ', in_top_5/len(scraped_subs_dict.keys()))
    print('recallrate at 10: ', in_top_10/len(scraped_subs_dict.keys()))
    print('mean average precision: ', np.mean(ave_p))
    print('formatted')
    print(round(np.mean(ave_p), 3),' & ',round(np.mean(rank_scores), 3),' & ',round(in_top_5/len(scraped_subs_dict.keys()), 3),' & ',round(in_top_10/len(scraped_subs_dict.keys()), 3))
    return round(np.mean(ave_p), 3)


In [152]:
subs_df =pd.read_csv('../data/Food_substitution_gt1.tsv', sep='\t')
subs_df.head()

,Food id,Food,Verified Subsititions,Food substitutions,Similar Food
0,usda#09229,"Papaya nectar, canned","['Peach nectar, canned, with sucralose', 'Appl...","['Oil, almond', 'SILK Peach soy yogurt', 'Ham,...","['Pickles, cucumber, sweet ', 'Peach nectar, c..."
1,usda#09206,"Orange juice, raw","['Mangos, raw', 'Orange juice, chilled, includ...","['Acerola juice, raw', 'Milk, filled, fluid', ...","['Onions, spring or scallions, raw', 'Cauliflo..."
2,usda#09008,"Apples, canned, sweetened","['Peach nectar, canned, with sucralose', 'Pome...","['Celery, raw', 'Peaches, yellow, raw', 'Dates...","['Pickles, cucumber, sweet ', 'Peach nectar, c..."
3,usda#04529,"Oil, almond","['Oil, sunflower, linoleic', 'Oil, cottonseed,...","['Milk, dry, whole', 'SILK Coffee, soymilk', '...","['Oil, sunflower, linoleic', 'Oil, cottonseed,..."
4,usda#07022,"Frankfurter, beef, unheated","['Frankfurter, meat and poultry, unheated', 'S...","['SILK Coffee, soymilk', 'Sauce, peppers, hot'...","['Oil, canola', 'Oil, peanut, salad or cooking..."


In [153]:
subs2_df =pd.read_csv('../data/Food_substitution_gt2.csv')
subs2_df.head()

,Food id,Food,Food Short,Verified Food substitutions,Similar Food
0,usda#11090,"Broccoli, raw",broccoli,"['Squash, summer, all varieties', 'Mustard gre...","['Squash, summer, all varieties', 'Mustard gre..."
1,usda#01040,"Cheese, swiss",swiss cheese,"['Cheese, fontina', 'Cheese, low-sodium, chedd...","['Cheese, fontina', 'Cheese, low-sodium, chedd..."
2,usda#11445,"Seaweed, kelp, raw",kelp,"['Seaweed, irishmoss, raw', ""Jew's ear, raw"", ...","['Seaweed, irishmoss, raw', ""Jew's ear, raw"", ..."
3,usda#15065,"Fish, pollock, Atlantic",pollock,"['Fish, wolffish, Atlantic', 'Fish, salmon, At...","['Fish, wolffish, Atlantic', 'Fish, salmon, At..."
4,usda#20076,"Wheat, durum",wheat,"['Wheat, soft red winter', 'Wheat, soft white'...","['Wheat, soft red winter', 'Wheat, soft white'..."


In [154]:
df_usda = pd.read_csv('../data/usda-labels.csv')
df_usda.head()

,str,Resource
0,"Butter, salted",http://idea.rpi.edu/heals/kb/usda#01001
1,"Butter, whipped, with salt",http://idea.rpi.edu/heals/kb/usda#01002
2,"Butter oil, anhydrous",http://idea.rpi.edu/heals/kb/usda#01003
3,"Cheese, blue",http://idea.rpi.edu/heals/kb/usda#01004
4,"Cheese, brick",http://idea.rpi.edu/heals/kb/usda#01005


In [155]:
food_label_map ={}
food_id_map = {}
for label, res in df_usda[['str','Resource']].values:
    #print(label,res)
    if res not in food_label_map:
        food_label_map[res] = label
        food_id_map[label.strip()] = res

In [173]:
scraped_subs_dict = dict()
for i,row in subs_df.iterrows():
    #print (ast.literal_eval(row[2]))
    #print  (food_id_map[row[1]])
    subs_list = [ food_id_map[s.strip()] for s in ast.literal_eval(row[2]) ]
    if food_id_map[row[1]] in word_vectors:
        scraped_subs_dict[food_id_map[row[1]]]= set(subs_list)

for i,row in subs2_df.iterrows():
    #print (ast.literal_eval(row[2]))
    #print  (food_id_map[row[1]])
    subs_list = [ food_id_map[s.strip()] for s in ast.literal_eval(row[3]) ]
    if food_id_map[row[1]] in word_vectors:
        if food_id_map[row[1]] not in scraped_subs_dict:
            scraped_subs_dict[food_id_map[row[1]]]= set(subs_list)
        else:
            scraped_subs_dict[food_id_map[row[1]]].union(set(subs_list))
    else:
        print (food_id_map[row[1]], row[1], 'not in embeddings')
        


In [174]:
list(scraped_subs_dict.items())[:1]

[('http://idea.rpi.edu/heals/kb/usda#09229',
  {'http://idea.rpi.edu/heals/kb/usda#09008',
   'http://idea.rpi.edu/heals/kb/usda#09016',
   'http://idea.rpi.edu/heals/kb/usda#09020',
   'http://idea.rpi.edu/heals/kb/usda#09079',
   'http://idea.rpi.edu/heals/kb/usda#09163',
   'http://idea.rpi.edu/heals/kb/usda#09238',
   'http://idea.rpi.edu/heals/kb/usda#09240',
   'http://idea.rpi.edu/heals/kb/usda#09241',
   'http://idea.rpi.edu/heals/kb/usda#09251',
   'http://idea.rpi.edu/heals/kb/usda#09254',
   'http://idea.rpi.edu/heals/kb/usda#09256',
   'http://idea.rpi.edu/heals/kb/usda#09257',
   'http://idea.rpi.edu/heals/kb/usda#09262',
   'http://idea.rpi.edu/heals/kb/usda#09351',
   'http://idea.rpi.edu/heals/kb/usda#09370',
   'http://idea.rpi.edu/heals/kb/usda#09374',
   'http://idea.rpi.edu/heals/kb/usda#09379',
   'http://idea.rpi.edu/heals/kb/usda#09442',
   'http://idea.rpi.edu/heals/kb/usda#09448'})]

In [175]:
mrr_map_new(scraped_subs_dict,1)

number of ings:  168
mean reciprocal rank:  0.6366380024387187
recallrate at 5:  0.7440476190476191
recallrate at 10:  0.7857142857142857
mean average precision:  0.48635624721189546
formatted
0.486  &  0.637  &  0.744  &  0.786


0.486

In [49]:
mrr_map_new(scraped_subs_dict,2)

number of ings:  105
mean reciprocal rank:  0.39658383011324183
recallrate at 5:  0.6190476190476191
recallrate at 10:  0.7238095238095238
mean average precision:  0.43368992885628466
formatted
0.434  &  0.397  &  0.619  &  0.724


0.434